In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

from PIL import Image
from io import BytesIO
import requests

Problem 1

In [ ]:
# URL of the image
image_url = 'https://raw.githubusercontent.com/MurpheyLab/ME455_public/main/figs/lincoln.jpg'

# Fetch the image data from the URL
response = requests.get(image_url)

# Create a BytesIO object from the response data
image_data = BytesIO(response.content)

# Open the image using PIL
image = Image.open(image_data)

# Convert the image to a NumPy array
image_array = np.array(image)
image_array = np.flip(image_array, axis=0)

print('image_array.shape: ', image_array.shape)

plt.imshow(image_array, origin='lower', cmap='gray') # note that for "imshow" the origin of the coordinate is at top left instead of bottom left
plt.show()
plt.close()

In [ ]:
xgrids = np.linspace(0.0, 1.0, image_array.shape[0])  # the x coordinates of image pixels in the new space
dx = xgrids[1] - xgrids[0]
ygrids = np.linspace(0.0, 1.0, image_array.shape[1])  # the y coordinates of image pixels in the new space
dy = ygrids[1] - ygrids[0]

# we now invert dark and light pixel density and normalize the density values so it is a valid probability distribution
density_array = 255.0 - image_array  # we want higher density at darker regions
density_array /= np.sum(density_array) * dx * dy  # so the integral is 1

def image_density(s):
    """ Continuous density function based on the image
    Inputs:
        s - a numpy array containing the (x,y) coordinate within the 1m-by-1m space
    Return:
        val - the density value at s
    """
    s_x, s_y = s

    # Find the pixel closest to s in the 1-by-1 space
    # Note that in image the first pixel coordinate correspond to the y-axis in the 1-by-1 space
    pixel_idx_y = np.argmin(np.abs(xgrids - s_x))
    pixel_idx_x = np.argmin(np.abs(ygrids - s_y))

   # the density at s is the same as the closest pixel density
    val = density_array[pixel_idx_x, pixel_idx_y]

    return val

Proposal distribution: Uniform

In [ ]:
num_samples = 10000
gsamples = np.random.uniform(low=0.0, high=1.0, size=(num_samples, 2))
scalars = np.random.uniform(low=0.0, high=1.0, size=num_samples)
samples = np.zeros((num_samples,2))
weights = np.zeros((num_samples,))
M = 1.0

for i in range(num_samples):
    if image_density(gsamples[i]) > M * scalars[i]:
        samples[i] = gsamples[i]
        weights[i] = image_density(gsamples[i])

weights /= np.max(weights)

fig, ax = plt.subplots(1, 1, figsize=(4,4), dpi=120, tight_layout=True)
ax.set_aspect('equal')
ax.set_xlim(0.0, 1.0)
ax.set_ylim(0.0, 1.0)

for sample,weight in zip(samples, weights):
    ax.plot(sample[0], sample[1], linestyle='', marker='o', markersize=2, color='k', alpha=weight)

plt.show()
plt.close()

Proposal distribution: Gaussian

In [ ]:
num_samples = 10000
gsamples = np.random.normal(loc=0.5, scale=0.2, size=(num_samples, 2))
scalars = np.random.normal(loc=0.5, scale=0.1, size=num_samples)
samples = np.zeros((num_samples,2))
weights = np.zeros((num_samples,))
M = 1.0

for i in range(num_samples):
    if image_density(gsamples[i]) > M * scalars[i]:
        samples[i] = gsamples[i]
        weights[i] = image_density(gsamples[i])

weights /= np.max(weights)

fig, ax = plt.subplots(1, 1, figsize=(4,4), dpi=120, tight_layout=True)
ax.set_aspect('equal')
ax.set_xlim(0.0, 1.0)
ax.set_ylim(0.0, 1.0)

for sample,weight in zip(samples, weights):
    ax.plot(sample[0], sample[1], linestyle='', marker='o', markersize=2, color='k', alpha=weight)

plt.show()
plt.close()

Problem 2

In [ ]:
# Predefined list of colors
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']

# Create a cycle iterator for colors
color_cycle = itertools.cycle(colors)

In [ ]:
def get_velocity(state, u):
    return [np.cos(state[2]) * u[0], np.sin(state[2]) * u[0], u[1]]

def get_measurement(state, sigma_measurement):
    copy_state = state.copy()
    return [copy_state[0] + np.random.normal(0, sigma_measurement),
            copy_state[1] + np.random.normal(0, sigma_measurement),
            copy_state[2] + np.random.normal(0, sigma_measurement)]

def take_samples(state, sigma_measurement):
    samples = []
    for i in range(num_samples):
        samples.append(get_measurement(state, sigma_measurement))
    return samples

def update_state(state, u, sigma_process):
    new_state = state.copy()
    new_state[0] += np.cos(state[2]) * u[0] * dt + np.random.normal(0, sigma_process)
    new_state[1] += np.sin(state[2]) * u[0] * dt + np.random.normal(0, sigma_process)
    new_state[2] += u[1] * dt + np.random.normal(0, sigma_process)
    return new_state

In [ ]:
# start with taking a bunch of measurements from a gaussian distribution with mean
# 0 and sigma_measurement.
u = [1, -0.5]
state = [0, 0, np.pi/2] # x, y, theta
total_time = 2 * np.pi
dt = 0.1

num_samples = 100
sigma_process = 0.02
sigma_measurement = 0.2
time_steps = int(total_time / dt)

estimated_state = state

positions = [state[:2]]
estimate_positions = [state[:2]]
for t in range(time_steps):
    weights = np.ones((num_samples,)) / num_samples
    samples = take_samples(state, sigma_measurement)
    predictions = []
    for sample in samples:
        predictions.append(update_state(sample, u, sigma_process))
    # once we have our belief, we execute u
    state = update_state(state, u, sigma_process)
    positions.append(state)
    # now we receive a measurement z_{t+1}
    z = get_measurement(state, sigma_measurement)
    weights = 1/np.sqrt(2*np.pi*sigma_measurement**2) * np.exp(-0.5 * np.sum((np.array(z) - np.array(predictions))**2, axis=1) / sigma_measurement**2)
    normalized_weights = weights / np.sum(weights)
    estimated_state = np.sum(np.array(predictions) * normalized_weights[:, np.newaxis], axis=0)
    estimate_positions.append(estimated_state[:2])
    # resample
    resample_indices = np.random.choice(np.arange(num_samples), num_samples, p=normalized_weights)
    samples = np.array(samples)
    samples = samples[resample_indices]
    plt.plot([position[0] for position in positions], [position[1] for position in positions], linestyle='-', marker='o', markersize=2, color='k')
    plt.plot([position[0] for position in estimate_positions], [position[1] for position in estimate_positions], linestyle='-', marker='o', markersize=2, color='r')
    if t % 10 == 0:
        for sample, weight in zip(samples, normalized_weights):
            plt.plot(sample[0], sample[1], linestyle='', marker='o', markersize=2, color='b', alpha=weight)  # Set alpha based on weight
        plt.xlim(-2, 5)
        plt.ylim(-2.5, 2.5)
    plt.draw()
    plt.pause(0.1)

Problem 3

In [ ]:
def generate_samples(w1, w2, w3, mew1, mew2, mew3, sigma1, sigma2, sigma3, num_samples):
    samples = []
    for i in num_samples:
        samples.append(np.random.normal(mew1, sigma1, size=2) * w1
                       + np.random.normal(mew2, sigma2, size=2) * w2
                       + np.random.normal(mew3, sigma3, size=2) * w3)
    return samples

In [ ]:
w1 = 0.5
w2 = 0.2
w3 = 0.3

mew1 = np.array([0.35, 0.38])
mew2 = np.array([0.68, 0.25])
mew3 = np.array([0.56, 0.64])

sigma1 = np.array([[0.01, 0.004],[0.004, 0.01]])
sigma2 = np.array([[0.005, -0.003],[-0.003, 0.005]])
sigma3 = np.array([[0.008, 0.0], [0.0, 0.004]])

num_samples = 100

samples = generate_samples(w1, w2, w3, mew1, mew2, mew3, sigma1, sigma2, sigma3, num_samples)